In [1]:
!c:\Python312\python.exe -m pip install openai python-dotenv gradio

  Using cached openai-2.1.0-py3-none-any.whl.metadata (29 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached gradio-5.49.0-py3-none-any.whl.metadata (16 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.10-py3-none-any.whl.metadata (68 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.118.0-py3-none-any.whl.metadata (28 kB)
  Using cached ffmpy-0.6.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.13.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached huggingface_hub-0.35.3-py3-none-any.whl.metadata (14 kB)
  Using cached j

ERROR: Could not install packages due to an OSError: [WinError 2] El sistema no puede encontrar el archivo especificado: 'c:\\Python312\\Scripts\\websockets.exe' -> 'c:\\Python312\\Scripts\\websockets.exe.deleteme'


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: c:\Python312\python.exe -m pip install --upgrade pip


In [37]:
pip install --upgrade openai


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
from dotenv import load_dotenv
import os, time, gradio as gr
from openai import OpenAI
import requests

In [52]:
load_dotenv(override=True)

True

In [53]:
open_router_api_key = os.getenv('OPEN_ROUTER_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

In [61]:
open_router_ai = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=open_router_api_key)
groq_ai = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")

### Diccionario de modelos por proveedor y tarea 

In [63]:
MODELOS = {
    "OpenRouter": {
        "Traducción": ["gpt-3.5-turbo", "deepseek/deepseek-r1:free"],
        "Resumen": ["gpt-4o-mini", "meta-llama/llama-3.1-8b-instruct:free"]
    },
    "Gemini": {
        "Traducción": ["gemini-2.0-flash", "gemini-2.5-flash"],
        "Resumen": ["gemini-2.0-pro", "gemini-2.5-pro"]
    },
    "Groq": {
        "Traducción": ["llama-3.3-70b-versatile"], 
        "Resumen": ["llama-3.3-70b-versatile"]      
    },
    "Ollama": {
        "Traducción": ["llama3.2"],
        "Resumen": ["mistral"]
    }
}


### Procesar texto

In [64]:
def procesar_texto(proveedor, tarea, modelo, texto):
    if not texto.strip():
        return "Por favor, ingresa un texto.", ""

    start_time = time.time()
    prompt = ""

    if tarea == "Traducción":
        prompt = f"Traduce el siguiente texto al español:\n\n{texto}"
    elif tarea == "Resumen":
        prompt = f"Resume el siguiente texto en pocas líneas:\n\n{texto}"

    try:
        # --- OPENROUTER ---
        if proveedor == "OpenRouter":
            resp = open_router_ai.chat.completions.create(
                model=modelo,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=512
            )
            result = resp.choices[0].message.content

        # --- GEMINI ---
        elif proveedor == "Gemini":
            gemini = OpenAI(
                api_key=gemini_api_key,
                base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
            )

            resp = gemini.chat.completions.create(
                model=modelo,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=512
            )
            result = resp.choices[0].message.content

        # --- GROQ ---
        elif proveedor == "Groq":
            resp = groq_ai.chat.completions.create(
                model=modelo,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=512
            )
            result = resp.choices[0].message.content

        # --- OLLAMA ---
        elif proveedor == "Ollama":
            response = requests.post(
                "http://localhost:11434/api/generate",
                json={"model": modelo, "prompt": prompt},
                timeout=60
            )
            result = ""
            for line in response.iter_lines():
                if line:
                    data = line.decode("utf-8")
                    if '"response":"' in data:
                        part = data.split('"response":"')[1].split('"')[0]
                        result += part

        else:
            result = "Proveedor no válido."

        end_time = time.time()
        elapsed = round(end_time - start_time, 2)
        return result, f"Tiempo de inferencia: {elapsed}s"

    except Exception as e:
        # 🧠 Esta parte te mostrará qué proveedor falló y el detalle del error
        error_detallado = f"⚠️ Error en {proveedor}: {str(e)}\n\n{traceback.format_exc()}"
        return error_detallado, 

### Interfaz GRADIO

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 💬 App Multi-IA (OpenRouter + Gemini + Groq + Ollama)")

    proveedor = gr.Dropdown(list(MODELOS.keys()), label="Proveedor")
    tarea = gr.Dropdown(["Traducción", "Resumen"], label="Tarea")
    modelo = gr.Dropdown([], label="Modelo")

    texto = gr.Textbox(label="Texto de entrada", placeholder="Escribe tu texto aquí...")
    salida = gr.Textbox(label="Resultado", lines=6)
    tiempo = gr.Textbox(label="Tiempo de inferencia")

    def actualizar_modelos(proveedor, tarea):
        if proveedor and tarea:
            modelos = MODELOS[proveedor][tarea]
            return gr.update(choices=modelos, value=modelos[0])
        return gr.update(choices=[], value=None)

    proveedor.change(fn=actualizar_modelos, inputs=[proveedor, tarea], outputs=modelo)
    tarea.change(fn=actualizar_modelos, inputs=[proveedor, tarea], outputs=modelo)

    boton = gr.Button("Ejecutar")
    boton.click(procesar_texto, inputs=[proveedor, tarea, modelo, texto], outputs=[salida, tiempo])

demo.launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Maria Camila\AppData\Local\Temp\ipykernel_5544\3806728639.py", line 16, in procesar_texto
    resp = open_router_ai.chat.completions.create(
        model=modelo,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=512
    )
  File "C:\Users\Maria Camila\AppData\Roaming\Python\Python313\site-packages\openai\_utils\_utils.py", line 286, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\Maria Camila\AppData\Roaming\Python\Python313\site-packages\openai\resources\chat\completions\completions.py", line 1147, in create
    return self._post(
           ~~~~~~~~~~^
        "/chat/completions",
        ^^^^^^^^^^^^^^^^^^^^
    ...<46 lines>...
        stream_cls=Stream[ChatCompletionChunk],
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "C:\Users\Maria Camila\AppData\Roaming\Python\Python313\site-packages\openai\_base_client.py", line 1259, in post
    return cast(ResponseT, self.request(